# Abstract

<p style="font-size:16px; line-height:1.6;">
This project presents a comprehensive algorithmic trading ecosystem developed in Python, combining quantitative strategy formulation with a robust, modular software architecture. The core investment vehicle is a <strong>"High-Quality Momentum" (HQM)</strong> strategy, which selects the top 50 S&amp;P 500 assets based on a composite score of 1-month, 3-month, 6-month, and 1-year return percentiles. In addition to HQM strategy, a simple version of Relative Strength Index (RSI) strategy is implemented to construct a comparison mechanism, giving the user flexibility over which strategy to use based on their backtest performance. 
</p>

<p style="font-size:16px; line-height:1.6;">
Supporting this strategy is a custom-built trading engine designed with Object-Oriented Programming (OOP) principles, featuring distinct classes for market data querying, order management (Market and Limit orders), and broker simulation. To ensure system reliability, the engine is validated through a rigorous unit testing suite using <code>pytest</code>. Risk management is enforced via a pre-trade volatility filter—excluding the top decile of most volatile stocks—and a reactive 20% drawdown stop-loss. The system’s performance is validated through historical backtesting against the SPY benchmark, evaluating key metrics such as CAGR, Sharpe Ratio, and volatility to demonstrate both financial viability and technical stability.
</p>

<p style="font-size:16px; line-height:1.6;">
The projects aims to extract S&amp;P 500 Index and utilize both HQM and RSI strategies to produce two separate trading strategies and write the strategies into two separate .xlsx file, namely momentum_strategy.xlsx and rsi_strategy.xlsx. Then a few plots will be generated to visualize backtest result supported by the backtest code snippet. Thus, any user can have a trading strategy that is backtested and risk-managed stored in an excel sheet with clear headers and cash amount to be invested, given a certain amount of initial portfolio value.
</p>

# Project Overview

<p style="font-size:16px; line-height:1.6;">
<strong>Project Name:</strong> Algorithmic Trading
</p>

<p style="font-size:16px; line-height:1.6;">
<strong>Team Members:</strong> Sean Wang, David Liu, Timothy Lu
</p>

<p style="font-size:16px; line-height:1.6;">
<strong>Report Date:</strong> 12/2/2025
</p>

# Final Report

In [1]:
# ==========================================
# First Step: 
# Install all required libraries and dependencies
# ========================================== 

%pip install numpy pandas requests scipy xlsxwriter yfinance datetime matplotlib openpyxl lxml

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## Results and Performance Analysis

### Performance Overview
The quantitative backtest, conducted from **January 2024 to January 2025**, reveals that while all implemented strategies were profitable, they underperformed the passive market benchmark during this specific period. 

* **Benchmark Performance:** The **S&P 500 Benchmark (SPY)** delivered the strongest performance, achieving a Compound Annual Growth Rate (CAGR) of **25.69%** with a high Sharpe Ratio of **1.88**.

In comparison, the algorithmic strategies yielded more modest results:

* **RSI Strategy:** This was the top-performing algorithmic approach, generating a **14.36% CAGR** with a Sharpe Ratio of **1.04**. Its lower volatility (**11.86%**) compared to the benchmark (12.58%) indicates it successfully identified stable entry points, though it captured less total upside.
* **High-Quality Momentum (HQM) Strategy:** This strategy lagged significantly, returning **11.84% CAGR** with the highest volatility of the group (**16.16%**) and a Sharpe Ratio of **0.61**.

### Impact of Risk Management Protocols
A critical observation from the data is that the performance metrics for the "Stop-Loss" portfolios are **identical** to their unrestricted counterparts (e.g., HQM at 11.84% is equal to HQM w/ Stop-Loss at 11.84%). 

This indicates that the **20% portfolio drawdown trigger was never activated** during the backtesting window. The market environment in 2024 was characterized by a consistent uptrend with shallow corrections; consequently, the portfolio never experienced a peak-to-trough decline severe enough to trigger the defensive exit to cash. While the insurance policy was in place, the market conditions did not necessitate its use.

### Discussion: The "Bull Run" Effect
The underperformance of the HQM strategy relative to the SPY is likely attributable to the **market weighting mechanism**. 
* **Cap-Weighted Benchmark:** The S&P 500 is capitalization-weighted, meaning its 2024 returns were heavily driven by a small handful of mega-cap technology companies (e.g., NVIDIA, Meta) that saw explosive growth. 
* **Equal-Weighted Strategy:** Our strategy utilizes an **equal-weighted allocation** (investing the same amount into each of the top 50 stocks). In a market where the "giants" outperform the average stock, an equal-weighted strategy will mathematically trail the cap-weighted index, even if the stock selection logic is sound.

---

## Conclusion and Future Outlook

### Technical Validation
Despite trailing the benchmark in raw returns, the project stands as a successful technical implementation of a robust algorithmic trading engine. The system demonstrated the ability to:
1.  Autonomously scrape constituent data.
2.  Calculate complex technical indicators (Momentum Percentiles and RSI).
3.  Simulate portfolio management logic including volatility filtering and conditional stop-losses. 

The fact that the RSI strategy achieved a **Sharpe Ratio above 1.0** demonstrates that the system can identify risk-adjusted value, even if it missed the extreme outliers of the benchmark.

### Recommendations for Improvement
To bridge the performance gap in future iterations, we recommend three key adjustments:

1.  **Dynamic Rebalancing:** Transitioning from a static one-time purchase to a monthly rebalancing schedule would allow the Momentum strategy to "ride the winners" and cut losers, potentially capturing more of the mega-cap upside.
2.  **Adaptive Risk Thresholds:** Since the 20% stop-loss was too loose to be useful in this bull market, implementing a tighter, trailing stop-loss (e.g., 10%) or a volatility-adjusted stop (e.g., 2x ATR) might have actively protected profits during minor dips.
3.  **Regime-Based Weighting:** Introducing a logic to shift between "Equal Weight" and "Market-Cap Weight" based on market breadth could allow the algorithm to mimic the benchmark during top-heavy rallies while retaining diversification benefits during normal markets.

In [10]:
# ==========================================
# 1. HQM Strategy
# ==========================================

import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter
import yfinance as yf
import io
from statistics import mean
from scipy.stats import percentileofscore as score
from datetime import date, timedelta

# --- STEP 1: SCRAPE WIKIPEDIA ---
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
header = {
    'User-Agent': "Mozilla/5.0"
}

response = requests.get(url, headers=header)
payload = pd.read_html(io.StringIO(response.text))

sp500_table = None
for table in payload:
    if 'Symbol' in table.columns:
        sp500_table = table
        break

if sp500_table is None:
    print("Error: Could not find the S&P 500 table on Wikipedia.")
    fresh_tickers = []
else:
    fresh_tickers = sp500_table['Symbol'].str.replace('.', '-', regex=False).tolist()
    print(f'Successfully loaded {len(fresh_tickers)} live tickers from Wikipedia.')


# --- STEP 2: PORTFOLIO INPUT ---
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio:')

    try:
        float(portfolio_size)
    except ValueError:
        print('That is not a number! \nPlease try again:')
        portfolio_size = input('Enter the size of your portfolio:')


portfolio_input()

# --- STEP 3: BUILD HQM STRATEGY ---
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]
hqm_rows = []

for symbol in fresh_tickers:
    try:
        end_date = date.today()
        start_date = end_date - timedelta(days=2 * 365)

        df = yf.download(symbol, start=start_date, end=end_date, progress=False, auto_adjust=True)

        if df.empty: continue

        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)

        last_trading_date = df.index[-1]
        current_price = float(df['Close'].iloc[-1])

        date_1y_target = last_trading_date - pd.DateOffset(years=1)
        date_6m_target = last_trading_date - pd.DateOffset(months=6)
        date_3m_target = last_trading_date - pd.DateOffset(months=3)
        date_1m_target = last_trading_date - pd.DateOffset(months=1)

        try:
            price_1y = float(df[:date_1y_target]['Close'].iloc[-1])
            price_6m = float(df[:date_6m_target]['Close'].iloc[-1])
            price_3m = float(df[:date_3m_target]['Close'].iloc[-1])
            price_1m = float(df[:date_1m_target]['Close'].iloc[-1])

        except IndexError:
            continue

        hqm_rows.append([
            symbol,
            current_price,
            'N/A',
            (current_price - price_1y) / price_1y,
            'N/A',
            (current_price - price_6m) / price_6m,
            'N/A',
            (current_price - price_3m) / price_3m,
            'N/A',
            (current_price - price_1m) / price_1m,
            'N/A',
            'N/A'
        ])

    except Exception:
        pass

hqm_df = pd.DataFrame(hqm_rows, columns=hqm_columns)
print(f'Total stocks processed successfully: {len(hqm_df)}')

# --- STEP 4: CALCULATE PERCENTILES ---
time_periods = ['One-Year', 'Six-Month', 'Three-Month', 'One-Month']

for row in hqm_df.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_df.loc[row, percentile_col] = score(hqm_df[change_col], hqm_df.loc[row, change_col]) / 100

# --- STEP 5: CALCULATE HQM SCORE ---
for row in hqm_df.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(float(hqm_df.loc[row, f'{time_period} Return Percentile']))
    hqm_df.loc[row, 'HQM Score'] = mean(momentum_percentiles)

# --- STEP 6: SELECT TOP CANDIDATES (Pre-Filter) ---
hqm_df.sort_values('HQM Score', ascending=False, inplace=True)
hqm_df = hqm_df[:70] # Take top 70 first, then filter by volatility down to 50
hqm_df.reset_index(drop=True, inplace=True)


# --- STEP 6.5: VOLATILITY FILTER (Risk Management) ---
print("Applying Volatility Filter: removing highest 10% volatility stocks...")

vol_list = []
for symbol in hqm_df['Ticker']:
    try:
        # Download 1 year of data for volatility calc
        df = yf.download(symbol, period='1y', progress=False, auto_adjust=True)
        if df.empty:
            continue
        
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)

        daily_ret = df['Close'].pct_change().dropna()
        if len(daily_ret) == 0:
            continue

        # Annualized Volatility
        vol = float(daily_ret.std()) * np.sqrt(252)
        vol_list.append([symbol, vol])

    except Exception as e:
        print(f"[Error] {symbol}: {e}")

vol_df = pd.DataFrame(vol_list, columns=['Ticker', 'Volatility'])

# Merge and Filter
hqm_df = hqm_df.merge(vol_df, on='Ticker', how='left')
hqm_df = hqm_df.dropna(subset=['Volatility'])

# Remove top 10% highest volatility
cut = hqm_df['Volatility'].quantile(0.90)
hqm_df = hqm_df[hqm_df['Volatility'] <= cut]

# Final Selection: Top 50
hqm_df = hqm_df[:50]
hqm_df.reset_index(drop=True, inplace=True)

print(f"Final stock count after volatility filter: {len(hqm_df)}")


# --- STEP 7: CALCULATE SHARES TO BUY ---
position_size = float(portfolio_size) / len(hqm_df.index)
for i in hqm_df.index:
    hqm_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_df.loc[i, 'Price'])

# --- STEP 8: EXCEL FORMATTING ---
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_df.to_excel(writer, sheet_name='Momentum Strategy', index=False)

background_color = '#0a0a23'
font_color = '#ffffff'
workbook = writer.book

string_template = workbook.add_format({'font_color': font_color, 'bg_color': background_color, 'border': 1})
dollar_template = workbook.add_format(
    {'num_format': '$0.00', 'font_color': font_color, 'bg_color': background_color, 'border': 1})
integer_template = workbook.add_format(
    {'num_format': '0', 'font_color': font_color, 'bg_color': background_color, 'border': 1})
percent_template = workbook.add_format(
    {'num_format': '0.0%', 'font_color': font_color, 'bg_color': background_color, 'border': 1})

column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['One-Year Price Return', percent_template],
    'E': ['One-Year Return Percentile', percent_template],
    'F': ['Six-Month Price Return', percent_template],
    'G': ['Six-Month Return Percentile', percent_template],
    'H': ['Three-Month Price Return', percent_template],
    'I': ['Three-Month Return Percentile', percent_template],
    'J': ['One-Month Price Return', percent_template],
    'K': ['One-Month Return Percentile', percent_template],
    'L': ['HQM Score', percent_template]
}

worksheet = writer.sheets['Momentum Strategy']

for column in column_formats.keys():
    worksheet.set_column(f'{column}:{column}', 25, column_formats[column][1])
    worksheet.write(f'{column}1', column_formats[column][0], column_formats[column][1])

writer.close()
print(" Momentum Strategy Completed and Saved!")

Successfully loaded 502 live tickers from Wikipedia.


Enter the size of your portfolio: 1000000


Total stocks processed successfully: 499
Applying Volatility Filter: removing highest 10% volatility stocks...
Final stock count after volatility filter: 50
 Momentum Strategy Completed and Saved!


The following chunk generates the HQM strategy as an excel sheet, stored under the same directory as the .ipynb file

Since the code is modified based on the HQM strategy code, the code faces the main issues as the HQM strategy code has.

In [11]:
# ==========================================
# 2. RSI Strategy 
# ==========================================

import numpy as np
import pandas as pd
import requests
import math
import io
import yfinance as yf
from datetime import date, timedelta
import xlsxwriter

# ==========================================
# STEP 1: SCRAPE WIKIPEDIA FOR S&P500 TICKERS
# ==========================================

url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
header = {
    'User-Agent': "Mozilla/5.0"
}

response = requests.get(url, headers=header)
payload = pd.read_html(io.StringIO(response.text))

sp500_table = None
for table in payload:
    if 'Symbol' in table.columns:
        sp500_table = table
        break

if sp500_table is None:
    print("Error: Could not find the S&P 500 table on Wikipedia.")
    fresh_tickers = []
else:
    fresh_tickers = sp500_table['Symbol'].str.replace('.', '-', regex=False).tolist()
    print(f"Successfully loaded {len(fresh_tickers)} live tickers from Wikipedia.")


# ==========================================
# STEP 2: PORTFOLIO INPUT
# ==========================================

def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the size of your portfolio:")

    try:
        float(portfolio_size)
    except ValueError:
        print("That is not a number! Please try again:")
        portfolio_size = input("Enter the size of your portfolio:")

portfolio_input()


# ==========================================
# STEP 3: BUILD RSI METRICS
# ==========================================

def compute_rsi(series, period=14):
    delta = series.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.ewm(span=period, adjust=False).mean()
    avg_loss = loss.ewm(span=period, adjust=False).mean()
    rs = avg_gain / avg_loss
    return 100 - (100 / (1 + rs))

rsi_rows = []

for symbol in fresh_tickers:
    try:
        end_date = date.today()
        start_date = end_date - timedelta(days=365)

        df = yf.download(symbol, start=start_date, end=end_date, progress=False, auto_adjust=True)

        if df.empty: 
            continue

        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)

        last_price = float(df['Close'].iloc[-1])
        rsi_series = compute_rsi(df['Close'])

        rsi_value = rsi_series.iloc[-1]

        rsi_rows.append([
            symbol,
            last_price,
            'N/A',
            rsi_value
        ])

    except Exception:
        pass

rsi_df = pd.DataFrame(rsi_rows, columns=[
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'RSI (14)'
])

print(f"Total stocks processed successfully: {len(rsi_df)}")

# ==========================================
# STEP 3.5: VOLATILITY FILTER (Risk Management)
# ==========================================
print("Applying Volatility Filter to RSI Strategy...")

vol_list = []
for symbol in rsi_df['Ticker']:
    try:
        df = yf.download(symbol, period='1y', progress=False, auto_adjust=True)
        if df.empty: continue
        
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)

        daily_ret = df['Close'].pct_change().dropna()
        if len(daily_ret) == 0: continue

        vol = float(daily_ret.std()) * np.sqrt(252)
        vol_list.append([symbol, vol])
    except:
        continue

vol_df = pd.DataFrame(vol_list, columns=['Ticker', 'Volatility'])
rsi_df = rsi_df.merge(vol_df, on='Ticker', how='left')
rsi_df = rsi_df.dropna(subset=['Volatility'])

# Remove top 10% highest volatility
cut = rsi_df['Volatility'].quantile(0.90)
rsi_df = rsi_df[rsi_df['Volatility'] <= cut]
print(f"Stocks remaining after volatility filter: {len(rsi_df)}")


# ==========================================
# STEP 4: FILTER STRATEGY (RSI < 30)
# ==========================================

rsi_df.sort_values('RSI (14)', ascending=True, inplace=True)
rsi_df = rsi_df[rsi_df['RSI (14)'] < 30]  # Oversold → long signal
rsi_df = rsi_df[:50]                     # take top 50
rsi_df.reset_index(drop=True, inplace=True)


# ==========================================
# STEP 5: CALCULATE SHARES TO BUY
# ==========================================

position_size = float(portfolio_size) / len(rsi_df.index)

for i in rsi_df.index:
    rsi_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rsi_df.loc[i, 'Price'])


# ==========================================
# STEP 6: EXCEL EXPORT (MATCHES YOUR HQM STYLE)
# ==========================================

writer = pd.ExcelWriter('rsi_strategy.xlsx', engine='xlsxwriter')
rsi_df.to_excel(writer, sheet_name='RSI Strategy', index=False)

background_color = '#0a0a23'
font_color = '#ffffff'
workbook = writer.book

string_template = workbook.add_format({'font_color': font_color, 'bg_color': background_color, 'border': 1})
dollar_template = workbook.add_format({'num_format': '$0.00', 'font_color': font_color, 'bg_color': background_color, 'border': 1})
integer_template = workbook.add_format({'num_format': '0', 'font_color': font_color, 'bg_color': background_color, 'border': 1})
number_template = workbook.add_format({'num_format': '0.0', 'font_color': font_color, 'bg_color': background_color, 'border': 1})

column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['RSI (14)', number_template]
}

worksheet = writer.sheets['RSI Strategy']

for column in column_formats.keys():
    worksheet.set_column(f'{column}:{column}', 25, column_formats[column][1])
    worksheet.write(f'{column}1', column_formats[column][0], column_formats[column][1])

writer.close()
print("RSI Strategy Completed and Saved!")

Successfully loaded 502 live tickers from Wikipedia.


Enter the size of your portfolio: 1000000


Total stocks processed successfully: 502
Applying Volatility Filter to RSI Strategy...
Stocks remaining after volatility filter: 451
RSI Strategy Completed and Saved!


The following chunk generates plots for backtest result visualization

Future Improvement: May incorporate more relative graphs into the code, for now only three most relative plots are included

# ==========================================
# 3. Backtest (Final Version - Fixed)
# ==========================================

import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt

# ---------------------------------------------------------
# 1. Load HQM Strategy Output
# ---------------------------------------------------------

file_path = "momentum_strategy.xlsx"
try:
    strategy = pd.read_excel(file_path)
    tickers = strategy["Ticker"].tolist()
    shares = strategy["Number of Shares to Buy"].tolist()
    print(f"Loaded {len(tickers)} tickers from momentum_strategy.xlsx")
except FileNotFoundError:
    print(f"ERROR: Could not find '{file_path}'")
    tickers, shares = [], []

# ---------------------------------------------------------
# Load RSI Strategy Output
# ---------------------------------------------------------

rsi_file_path = "rsi_strategy.xlsx"
try:
    rsi_strategy = pd.read_excel(rsi_file_path)
    rsi_tickers = rsi_strategy["Ticker"].tolist()
    rsi_shares = rsi_strategy["Number of Shares to Buy"].tolist()
    print(f"Loaded {len(rsi_tickers)} tickers from rsi_strategy.xlsx")
except FileNotFoundError:
    print(f"ERROR: Could not find '{rsi_file_path}'")
    rsi_tickers, rsi_shares = [], []

# ---------------------------------------------------------
# 2. Historical backtest window
# ---------------------------------------------------------
backtest_start = "2024-01-02"
backtest_end = "2025-01-02"
print("Backtest period:", backtest_start, "to", backtest_end)

# ---------------------------------------------------------
# 3. Safe price downloader
# ---------------------------------------------------------

def safe_download(tickers, start, end):
    if not tickers: return pd.DataFrame(), []
    
    # Force list if single ticker
    if isinstance(tickers, str): tickers = [tickers]

    raw = yf.download(
        tickers,
        start=start,
        end=end,
        progress=False,
        auto_adjust=True
    )

    # Handle yfinance MultiIndex columns (Price, Ticker)
    if isinstance(raw.columns, pd.MultiIndex):
        # Try to get 'Close', fallback to raw if structure differs
        try:
            raw = raw["Close"]
        except KeyError:
            pass

    raw = raw.ffill()

    # Remove completely missing tickers
    good = [t for t in tickers if t in raw.columns and raw[t].dropna().shape[0] > 0]
    return raw[good], good

# ---------------------------------------------------------
# 4. Download Price Data
# ---------------------------------------------------------

# HQM Data
prices, good_tickers = safe_download(tickers, backtest_start, backtest_end)
shares = [shares[tickers.index(t)] for t in good_tickers]
tickers = good_tickers

if len(tickers) == 0:
    raise ValueError("No valid HQM tickers available for backtest.")

# RSI Data
rsi_prices, rsi_good_tickers = safe_download(rsi_tickers, backtest_start, backtest_end)
rsi_shares = [rsi_shares[rsi_tickers.index(t)] for t in rsi_good_tickers]
rsi_tickers = rsi_good_tickers

if len(rsi_tickers) == 0:
    print("Warning: No valid RSI tickers available.")

# ---------------------------------------------------------
# 5. Compute Portfolio Values
# ---------------------------------------------------------

# HQM Portfolio
portfolio_value = (prices[tickers] * shares).sum(axis=1).to_frame("Portfolio")
# Safe normalization
start_val = float(portfolio_value["Portfolio"].iloc[0])
portfolio_norm = portfolio_value["Portfolio"] / start_val

# RSI Portfolio
if len(rsi_tickers) > 0:
    rsi_portfolio_value = (rsi_prices[rsi_tickers] * rsi_shares).sum(axis=1).to_frame("RSI_Portfolio")
    start_rsi = float(rsi_portfolio_value["RSI_Portfolio"].iloc[0])
    rsi_norm = rsi_portfolio_value["RSI_Portfolio"] / start_rsi
else:
    # Fallback
    rsi_portfolio_value = pd.DataFrame(index=portfolio_value.index)
    rsi_portfolio_value["RSI_Portfolio"] = portfolio_value["Portfolio"].iloc[0]
    rsi_norm = rsi_portfolio_value["RSI_Portfolio"] / rsi_portfolio_value["RSI_Portfolio"]

# ---------------------------------------------------------
# 6. Benchmark SPY
# ---------------------------------------------------------

spy_raw = yf.download("SPY", start=backtest_start, end=backtest_end, progress=False, auto_adjust=True)

# Robust SPY cleaning
if isinstance(spy_raw.columns, pd.MultiIndex):
    spy = spy_raw["Close"]
else:
    spy = spy_raw["Close"] if "Close" in spy_raw.columns else spy_raw

# Ensure it's a Series (single column), not a DataFrame
if isinstance(spy, pd.DataFrame):
    spy = spy.iloc[:, 0]

spy = spy.ffill()
spy_start = float(spy.iloc[0])
spy_norm = spy / spy_start

# ---------------------------------------------------------
# 7. Apply Stop-Loss (Risk Management)
# ---------------------------------------------------------

def apply_stop_loss(portfolio_series, limit=-0.20):
    # Ensure input is a Series
    if isinstance(portfolio_series, pd.DataFrame):
        portfolio_series = portfolio_series.iloc[:, 0]

    peak = portfolio_series.cummax()
    drawdown = (portfolio_series - peak) / peak
    
    with_stop = portfolio_series.copy()
    stop_triggered = False
    
    for i in range(len(with_stop)):
        if stop_triggered:
            with_stop.iloc[i] = with_stop.iloc[i-1] 
        elif drawdown.iloc[i] <= limit:
            stop_triggered = True
            with_stop.iloc[i] = with_stop.iloc[i] 
            
    return with_stop

# Apply to HQM
portfolio_value["Portfolio_Stop"] = apply_stop_loss(portfolio_value["Portfolio"], -0.20)

# Apply to RSI
rsi_portfolio_value["RSI_Stop"] = apply_stop_loss(rsi_portfolio_value["RSI_Portfolio"], -0.20)

# ---------------------------------------------------------
# 8. Performance Statistics Report (ROBUST VERSION)
# ---------------------------------------------------------

def get_metrics(series):
    if series.empty: return 0.0, 0.0, 0.0
    
    # 1. Force input to be a Pandas Series (fixes Ambiguity Error)
    if isinstance(series, pd.DataFrame):
        series = series.iloc[:, 0]
        
    # 2. Convert start/end to simple floats
    start = float(series.iloc[0])
    end = float(series.iloc[-1])
    
    days = (series.index[-1] - series.index[0]).days
    if days == 0: return 0.0, 0.0, 0.0
    
    years = days / 365.25
    cagr = (end / start) ** (1 / years) - 1
    
    daily_ret = series.pct_change().dropna()
    
    # 3. Force Volatility to be a simple float
    vol = float(daily_ret.std() * np.sqrt(252))
    
    sharpe = (cagr - 0.02) / vol if vol != 0 else 0.0
    return cagr, vol, sharpe

# Calculate metrics for all curves
m_spy = get_metrics(spy_norm)
m_hqm = get_metrics(portfolio_value["Portfolio"])
m_hqm_stop = get_metrics(portfolio_value["Portfolio_Stop"])
m_rsi = get_metrics(rsi_portfolio_value["RSI_Portfolio"])
m_rsi_stop = get_metrics(rsi_portfolio_value["RSI_Stop"])

print("\n" + "="*75)
print(f"{'STRATEGY PERFORMANCE REPORT':^75}")
print("="*75)
print(f"{'Strategy':<30} | {'CAGR':<10} | {'Volatility':<10} | {'Sharpe':<10}")
print("-" * 75)
print(f"{'1. SPY Benchmark':<30} | {m_spy[0]:.2%}    | {m_spy[1]:.2%}     | {m_spy[2]:.2f}")
print(f"{'2. HQM Strategy':<30} | {m_hqm[0]:.2%}    | {m_hqm[1]:.2%}     | {m_hqm[2]:.2f}")
print(f"{'3. HQM (w/ 20% Stop-Loss)':<30} | {m_hqm_stop[0]:.2%}    | {m_hqm_stop[1]:.2%}     | {m_hqm_stop[2]:.2f}")
print(f"{'4. RSI Strategy':<30} | {m_rsi[0]:.2%}    | {m_rsi[1]:.2%}     | {m_rsi[2]:.2f}")
print(f"{'5. RSI (w/ 20% Stop-Loss)':<30} | {m_rsi_stop[0]:.2%}    | {m_rsi_stop[1]:.2%}     | {m_rsi_stop[2]:.2f}")
print("="*75 + "\n")

# ---------------------------------------------------------
# 9. Plot Portfolio vs SPY vs RSI
# ---------------------------------------------------------
plt.figure(figsize=(12,6))

# Normalize inputs safely for plotting
def safe_norm(s):
    if isinstance(s, pd.DataFrame): s = s.iloc[:, 0]
    return s / float(s.iloc[0])

plt.plot(safe_norm(portfolio_value["Portfolio"]), label="HQM Strategy", alpha=0.5, linestyle='--')
plt.plot(safe_norm(rsi_portfolio_value["RSI_Portfolio"]), label="RSI Strategy", alpha=0.5, linestyle='--')

# Stop-Loss Versions (Thicker Lines)
plt.plot(safe_norm(portfolio_value["Portfolio_Stop"]), label="HQM (Stop-Loss)", linewidth=2)
plt.plot(safe_norm(rsi_portfolio_value["RSI_Stop"]), label="RSI (Stop-Loss)", linewidth=2)

# Benchmark
plt.plot(spy_norm, label="SPY Benchmark", color='black', alpha=0.6)

plt.title("Comparative Performance: HQM vs RSI vs SPY")
plt.ylabel("Normalized Return (Start = 1.0)")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# ---------------------------------------------------------
# 10. Drawdown Comparison
# ---------------------------------------------------------
plt.figure(figsize=(12,4))

# Helper for drawdown
def calc_dd(s):
    if isinstance(s, pd.DataFrame): s = s.iloc[:, 0]
    peak = s.cummax()
    return (s - peak) / peak

hqm_dd = calc_dd(portfolio_value["Portfolio"])
rsi_dd = calc_dd(rsi_portfolio_value["RSI_Portfolio"])

plt.plot(hqm_dd, color='red', label='HQM Drawdown', alpha=0.6)
plt.plot(rsi_dd, color='blue', label='RSI Drawdown', alpha=0.6)
plt.axhline(-0.20, color='black', linestyle='--', label='20% Stop Limit')

plt.title("Strategy Drawdown Comparison (%)")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [8]:
# ==========================================
# 4. Trading Engine
# ==========================================

import datetime
from datetime import date

try:
    import yfinance as yf
except ImportError:
    print("yfinance not installed. 'yahoo' source will not work.")
    yf = None

class MarketDataQuery:
    def __init__(self,
                 symbol: str,  # symbol means permno
                 time_frame: str,
                 start_date: str,
                 end_date: str,
                 frequency: str = '1d', # '1m', '2m', '5m', '15m', '30m', '60m', '90m', '1h', '1d', '5d', '1wk', '1mo', '3mo'
                 source: str = 'yahoo'): # default source is yahoo, it will ask yfinance to provide the data
        self.symbol = symbol
        self.time_frame = time_frame
        self.start_date = datetime.datetime.strptime(start_date, '%Y-%m-%d')  # convert string to datetime, opposite to strftime
        self.end_date = datetime.datetime.strptime(end_date, '%Y-%m-%d')
        self.frequency = frequency
        self.source = source

        self._validate()

    # you have to validate your query condition each time
    def _validate(self):
        if self.start_date > self.end_date:
            raise ValueError('Start date must be before end date')
        valid_frequencies = ['1m', '2m', '5m', '15m', '30m', '60m', '90m', '1h', '1d', '5d', '1wk', '1mo', '3mo']

        if self.frequency not in valid_frequencies:
            raise ValueError('Frequency must be one of {}'.format(valid_frequencies))

    def fetch(self):
        if self.source == 'test':

            freq_map = {'1m': 'T', # these are pandas alias
                        '2m': '2T',
                        '5m': '5T',
                        '15m': '15T',
                        '30m': '30T',
                        '60m': '60T',
                        '90m': '90T',
                        '1h': 'H',
                        '1d': 'D',
                        '5d': '5D',
                        '1wk': 'W',
                        '1mo': 'MS',
                        '3mo': '3MS'} # 'D'
            pd_freq = freq_map.get(self.frequency)
            if not pd_freq:
                raise ValueError('Test source does not support frequency: {self.frequency}')

            dates = pd.date_range(start=self.start_date, end=self.end_date, freq=pd_freq) # get() looks up value of self.frequency in freq_map
            if len(dates) == 0 and self.start_date <= self.end_date: # Handle case where start/end are same day
                dates = pd.to_datetime([self.start_date]) # set the start_date as dates

            prices = [100 + i*0.5 for i in range(len(dates))]  # add 0.5 each day

            df = pd.DataFrame({'date': dates, 'price': prices})
            return df

        elif self.source == 'yahoo':
            if yf is None:
                raise ImportError("yfinance is not installed. Cannot use 'yahoo' source.")

            df = yf.download( # input your parameter
                self.symbol,
                start=self.start_date.strftime('%Y-%m-%d'), # convert the datetime object to string, then send it to API
                end=self.end_date.strftime('%Y-%m-%d'),
                interval=self.frequency,
            )
            df.reset_index(inplace=True)
            return df

        else:
            raise ValueError(f'Unknown source: {self.source}')


# PriceBar is a container after getting the data
class PriceBar:
    def __init__(self, date, open_price, close_price, high_price, low_price, volume):
        self.date = date
        self.open = open_price
        self.close = close_price
        self.high = high_price
        self.low = low_price
        self.volume = volume

    def mid_price(self):
        return (self.high + self.low) / 2

    def is_bullish(self):
        return self.open < self.close

    def is_bearish(self):
        return self.open > self.close

    def __repr__(self):
        return (f'PriceBar(date={self.date}, open={self.open}, close={self.close}, '
                f'high={self.high}, low={self.low}, volume={self.volume})')


class TradeOrder:
    def __init__(self, symbol, side, quantity, order_type='market', price=None):
        self.symbol = symbol.upper()
        self.side = side.upper()  # 'BUY' or 'SELL'
        self.quantity = quantity
        self.order_type = order_type.lower()
        self.price = price
        # These are now set internally, not passed as args
        self.timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        self.status = 'pending'

    def cancel(self):
        if self.status == 'pending':
            self.status = 'cancelled'

    def __repr__(self):
        return (f'TradeOrder(symbol={self.symbol}, side={self.side}, quantity={self.quantity}, '
                f'type={self.order_type}, price={self.price}, status={self.status})')


class MarketOrder(TradeOrder):
    def __init__(self, symbol, side, quantity):
        super().__init__(symbol, side, quantity, order_type='market')

    def execute(self, market_price):
        self.price = market_price
        self.status = 'filled'


class LimitOrder(TradeOrder):
    def __init__(self, symbol, side, quantity, limit_price):
        super().__init__(symbol, side, quantity, order_type='limit', price=limit_price)

    def execute(self, market_price):
        if self.side == 'BUY' and market_price <= self.price:
            self.status = 'filled'
        elif self.side == 'SELL' and market_price >= self.price:
            self.status = 'filled'


class OrderReceipt:
    def __init__(self, symbol, side, order, timestamp, executed_price=None, executed_quantity=0, status='pending'):
        self.order_id = id(order)
        self.symbol = symbol.upper()
        self.side = side.upper()
        self.original_quantity = order.quantity  # order here is an object from TradeOrder
        self.executed_quantity = executed_quantity
        self.executed_price = executed_price
        self.timestamp = timestamp  # Use passed timestamp
        self.status = status

    def __repr__(self):
        return (f'OrderReceipt(symbol={self.symbol.upper()}, side={self.side.upper()}, '
                f'executed_qty={self.executed_quantity}/{self.original_quantity}, '
                f'executed_price={self.executed_price}, status={self.status}, '
                f'timestamp={self.timestamp})')


class IConnector:  # defines a contract for how your system talks to any broker.
    def getMarketData(self, symbol, start_date, end_date):
        raise NotImplementedError('Subclasses must implement getMarketData')

    def submitOrder(self, order):
        raise NotImplementedError('Subclasses must implement submitOrder')

    def getAccountInfo(self):
        raise NotImplementedError('Subclasses must implement getAccountInfo')


class MockBrokerConnector(IConnector):  # provides a dummybroker
    def __init__(self):
        self.cash_balance = 100000.0
        self.positions = {}  # e.g., {'AAPL': 100}
        self.order_history = []
        self.current_market_price = 100.0  # Added for predictable testing

    def getMarketData(self, symbol, start_date, end_date):
        dates = pd.date_range(start=start_date, end=end_date, freq='D')
        prices = [100 + i * 0.5 for i in range(len(dates))]
        df = pd.DataFrame({'date': dates, 'symbol': symbol, 'price': prices})
        return df

    def submitOrder(self, order):
        # Use the broker's current market price
        order.execute(self.current_market_price)
        self.order_history.append(order)

        if order.status == 'filled':
            if order.side == 'BUY':
                cost = order.price * order.quantity
                self.cash_balance -= cost
                self.positions[order.symbol] = self.positions.get(order.symbol, 0) + order.quantity
            elif order.side == 'SELL':
                revenue = order.price * order.quantity
                self.cash_balance += revenue
                self.positions[order.symbol] = self.positions.get(order.symbol, 0) - order.quantity

        # Return a receipt
        receipt = OrderReceipt(
            symbol=order.symbol,
            side=order.side,
            order=order,
            timestamp=datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            executed_price=order.price if order.status == 'filled' else None,
            executed_quantity=order.quantity if order.status == 'filled' else 0,
            status=order.status
        )
        return receipt

    def getAccountInfo(self):
        return {
            'cash_balance': self.cash_balance,
            'positions': self.positions,
            'order_history': self.order_history}

In [9]:
# ==========================================
# 5. Engine Test
# ==========================================
# ---------------------------------------------------------------
# 1. CONFIGURATION
# ---------------------------------------------------------------
tickers = ['WBD', 'APP', 'AMAT', 'GM', 'ALB', 'DLTR', 'TER', 'TPR', 'CAT', 'WDC', 'GOOG', 'EL', 'GOOGL', 'IVZ', 'NEM', 'DG', 'LUV', 'CMI', 'STX', 'INTC', 'STLD', 'GS', 'FOXA', 'JNJ', 'RL', 'JBHT', 'KEYS', 'C', 'DAL', 'GEV', 'ROST', 'NUE', 'EXPD', 'HII', 'CFG', 'CHRW', 'LRCX', 'ADI', 'FOX', 'GLW', 'ULTA', 'MS', 'SYF', 'MU', 'EXPE', 'KEY', 'WFC', 'CSCO', 'PH', 'F']
prices = [29.895, 692.18, 263.08, 81.52, 133.41, 128.585, 195.28, 122.24, 618.335, 174.625, 310.52, 105.17, 309.27, 26.835, 98.935, 131.755, 42.0, 517.76, 293.86, 39.225, 172.36, 907.85, 72.12, 210.97, 368.675, 200.065, 211.195, 112.15, 70.92, 666.2, 183.04, 165.51, 152.79, 329.67, 58.3, 158.6, 162.08, 281.353, 63.82, 90.745, 589.23, 180.085, 84.48, 244.35, 278.68, 20.76, 92.92, 78.68, 893.41, 13.81]
shares = [669, 28, 76, 245, 149, 155, 102, 163, 32, 114, 64, 190, 64, 745, 202, 151, 476, 38, 68, 509, 116, 22, 277, 94, 54, 99, 94, 178, 282, 30, 109, 120, 130, 60, 343, 126, 123, 71, 313, 220, 33, 111, 236, 81, 71, 963, 215, 254, 22, 1448]

strategy_df = pd.DataFrame({
    'Ticker': tickers,
    'Price': prices,
    'Number of Shares to Buy': shares
})

# ---------------------------------------------------------------
# 2. INITIALIZATION
# ---------------------------------------------------------------
print("Initializing Mock Broker...")
broker = MockBrokerConnector()
broker.cash_balance = 1000000.0

print(f"Starting Cash Balance: ${broker.cash_balance:,.2f}")
print("-" * 50)

# ---------------------------------------------------------------
# 3. EXECUTION
# ---------------------------------------------------------------
print(f"Processing {len(strategy_df)} orders...")

for index, row in strategy_df.iterrows():
    symbol = row['Ticker']
    qty = row['Number of Shares to Buy']
    price = row['Price']

    # Update Broker Price
    broker.current_market_price = price

    # Submit Order
    order = MarketOrder(symbol, 'buy', qty)
    receipt = broker.submitOrder(order)
    
    # Validation
    if receipt.status != 'filled':
        print(f"ERROR: Order for {symbol} failed!")
    
    # Print progress
    if index % 10 == 0:
        print(f"  Executed Trade {index+1}: Bought {qty} {symbol} @ ${price:.2f}")

print("-" * 50)

# ---------------------------------------------------------------
# 4. VERIFICATION
# ---------------------------------------------------------------
account_info = broker.getAccountInfo()
positions = account_info['positions']

# Verify Cash
expected_cost = sum([p * s for p, s in zip(prices, shares)])
expected_cash = 1000000.0 - expected_cost

print(f"Final Cash Balance:    ${account_info['cash_balance']:,.2f}")
print(f"Expected Cash Balance: ${expected_cash:,.2f}")

if abs(account_info['cash_balance'] - expected_cash) < 1.0:
    print("SUCCESS: Cash balance matches expected strategy cost.")
else:
    print("FAILURE: Cash balance calculation mismatch.")

# Verify Positions
print(f"Total Positions Held:  {len(positions)} (Expected: 50)")

if len(positions) == 50 and positions['WBD'] == 669 and positions['F'] == 1448:
     print("SUCCESS: All positions verified correctly.")
else:
     print("FAILURE: Position counts do not match.")

print("\nSimulation Complete.")

Initializing Mock Broker...
Starting Cash Balance: $1,000,000.00
--------------------------------------------------
Processing 50 orders...
  Executed Trade 1: Bought 669 WBD @ $29.89
  Executed Trade 11: Bought 64 GOOG @ $310.52
  Executed Trade 21: Bought 116 STLD @ $172.36
  Executed Trade 31: Bought 109 ROST @ $183.04
  Executed Trade 41: Bought 33 ULTA @ $589.23
--------------------------------------------------
Final Cash Balance:    $5,036.72
Expected Cash Balance: $5,036.72
SUCCESS: Cash balance matches expected strategy cost.
Total Positions Held:  50 (Expected: 50)
SUCCESS: All positions verified correctly.

Simulation Complete.
